In [2]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
cwd = os.getcwd()
path_rps = os.path.join(cwd, 'Dataset2')

In [4]:
training_datagen = ImageDataGenerator(rescale=1./255,
                                      validation_split=0.2)

In [5]:
train_generator = training_datagen.flow_from_directory(
        path_rps,
        target_size=(60, 80),
        batch_size=128,
        class_mode='binary',
        subset='training')

Found 71106 images belonging to 2 classes.


In [6]:
validation_generator = training_datagen.flow_from_directory(
        path_rps, 
        target_size=(60, 80),
        batch_size=128,
        class_mode='binary',
        subset='validation') 

Found 17776 images belonging to 2 classes.


In [7]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):

            if(logs.get('accuracy') > 0.85 and logs.get('val_accuracy') > 0.85):

                # Stop if threshold is met
                print("\nAccuracy and validation_accuracy are higher than 83%, so cancelling training!")
                self.model.stop_training = True

callbacks = myCallback()

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), 
                           activation='relu', input_shape=(60, 80, 3),strides=(1, 1) ),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.summary()

c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 58, 78, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 57, 77, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 55, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 27, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 25, 35, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 26112)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 250)            │     6,528,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           251 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,621,749 (25.26 MB)

 Trainable params: 6,621,749 (25.26 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(loss ='binary_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy'])

In [10]:
model.fit(train_generator,
          epochs=5,
          validation_data=validation_generator,
          # callbacks=[callbacks]
        )

Epoch 1/5


c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


556/556 ━━━━━━━━━━━━━━━━━━━━ 1315s 2s/step - accuracy: 0.8910 - loss: 0.2541 - val_accuracy: 0.9954 - val_loss: 0.0151
Epoch 2/5
556/556 ━━━━━━━━━━━━━━━━━━━━ 251s 450ms/step - accuracy: 0.9915 - loss: 0.0281 - val_accuracy: 0.9982 - val_loss: 0.0111
Epoch 3/5
556/556 ━━━━━━━━━━━━━━━━━━━━ 252s 453ms/step - accuracy: 0.9972 - loss: 0.0081 - val_accuracy: 0.9859 - val_loss: 0.0510
Epoch 4/5
556/556 ━━━━━━━━━━━━━━━━━━━━ 247s 444ms/step - accuracy: 0.9979 - loss: 0.0071 - val_accuracy: 0.9989 - val_loss: 0.0032
Epoch 5/5
556/556 ━━━━━━━━━━━━━━━━━━━━ 248s 446ms/step - accuracy: 0.9940 - loss: 0.0203 - val_accuracy: 0.9997 - val_loss: 9.4367e-04


In [12]:
saved_model_path = os.path.join(cwd, 'model.h5')
model.save(saved_model_path, save_format='tf')

In [14]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./